In [1]:
from efficientnet_lite import efficientnet_lite_params, build_efficientnet_lite

In [2]:
import time
import os, glob
import numpy as np
from PIL import Image
import torch
import torchvision.models as models
import torchvision.transforms as transforms

In [3]:
def load_checkpoint(net, checkpoint):
    from collections import OrderedDict

    temp = OrderedDict()
    if 'state_dict' in checkpoint:
        checkpoint = dict(checkpoint['state_dict'])
        # print(checkpoint)
    for k in checkpoint:
        # k2 = 'module.'+k if not k.startswith('module.') else k
        k2 = k.replace('module.', '') if k.startswith('module.') else k
        temp[k2] = checkpoint[k]

    net.load_state_dict(temp, strict=True)

In [4]:
def run_prediction(model, input_list):
    pred_result = []
    total_proc_time = 0.0
    total_model_time = 0.0
    for item in input_list:
        proc_s_time = time.time()
        # 이미지 불러오기
        image = Image.open(item).convert('RGB')

        # 전처리 수행
        input_tensor = preprocess(image)
        input_batch = input_tensor.unsqueeze(0)  # batch size = 1

        model_s_time = time.time()
        # 모델 추론
        with torch.no_grad():
            output = model(input_batch.to(device))
        # 예측 결과 출력
        _, predicted = torch.max(output.data, 1)
        model_r_time = time.time() - model_s_time
        proc_r_time = time.time() - proc_s_time

        total_proc_time += proc_r_time
        total_model_time += model_r_time
        pred_result.append(predicted.item())
    # print('Predicted:', predicted.item(), '|', 'Time: ', r_time, output, predicted)
    return {
        'result': pred_result,
        'n': len(pred_result),
        'total_proc_time': total_proc_time,
        'total_model_time': total_model_time,
        'avg_proc_time': total_proc_time / len(pred_result),
        'avg_model_time': total_model_time / len(pred_result),
    }

In [10]:
# 전처리 파이프라인
input_size = 224
CROP_PADDING = 32
MEAN_RGB = [0.498, 0.498, 0.498]
STDDEV_RGB = [0.502, 0.502, 0.502]

preprocess = transforms.Compose([
    transforms.Resize(input_size + CROP_PADDING, interpolation=Image.BICUBIC),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize(MEAN_RGB, STDDEV_RGB)
])

/opt/conda/lib/python3.8/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [11]:
use_gpu = False
num_classes = 2
model_name = 'efficientnet_lite0'
device = torch.device("cuda") if use_gpu else torch.device("cpu")


ckpt_path = f"./models/b64_lr5e-3_lite0/checkpoint-010000.pth.tar"
ckpt_selected = torch.load(ckpt_path, map_location=None if use_gpu else 'cpu')
model = build_efficientnet_lite(model_name, num_classes)
load_checkpoint(model, ckpt_selected)
model = model.to(device)
model.eval()
print('model loaded')

model loaded


In [19]:
# 이미지 불러오기
image = Image.open('../data/hair_verifying_all/positive/Female_O_L100.jpeg').convert('RGB')

# 전처리 수행
input_tensor = preprocess(image)
input_batch = input_tensor.unsqueeze(0)  # batch size = 1

s_time = time.time()
# 모델 추론
with torch.no_grad():
    output = model(input_batch.to(device))
# 예측 결과 출력
_, predicted = torch.max(output.data, 1)
r_time = time.time() - s_time
print('Predicted:', predicted.item(), '|', 'Time: ', r_time, output.numpy()[0], predicted.numpy()[0])

Predicted: 1 | Time:  0.025983095169067383 [-4.4621453  4.449293 ] 1
